In [2]:
!pip install langchain 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.4 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.2/456.2 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 371.2 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from langchain.llms import LlamaCpp
from langchain.chat_models import ChatOpenAI
from langchain.llms.base import LLM
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from typing import Any, List, Mapping, Optional
import requests
import json

# LM Studio runs an OpenAI-compatible API on port 3001 by default
LM_STUDIO_API_URL = "http://localhost:3001/v1"


In [15]:

class LMStudioLLM(LLM):
    """Custom LLM wrapper for LM Studio API."""
    
    model_name: str = "local-model"
    temperature: float = 0.7
    max_tokens: int = 512
    
    def _call(self, prompt: str, stop=None) -> str:
        """Call the LM Studio API."""
        try:
            # Format the request
            payload = {
                "model": self.model_name,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": self.temperature,
                "max_tokens": self.max_tokens
            }
            
            # Send request to LM Studio API
            response = requests.post(
                f"{LM_STUDIO_API_URL}/chat/completions",
                headers={"Content-Type": "application/json"},
                json=payload
            )
            
            response.raise_for_status()
            response_data = response.json()
            
            # Extract the generated text
            if "choices" in response_data and len(response_data["choices"]) > 0:
                return response_data["choices"][0]["message"]["content"]
            else:
                return "No response generated"
            
        except Exception as e:
            print(f"Error calling LM Studio API: {e}")
            return f"Error: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        """Return the type of LLM."""
        return "lm_studio"
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get identifying parameters."""
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }

# Create an instance of the LM Studio LLM
llm = LMStudioLLM(temperature=0.7, max_tokens=512)

# Test the LLM with a simple prompt
try:
    response = llm("Hello, how are you today?")
    print(f"LM Studio response: {response}")
except Exception as e:
    print(f"Error testing LM Studio connection: {e}")

# 🛠️ Analysis Prompt Template for Tool Output
analysis_prompt = PromptTemplate(
    input_variables=["tool_output", "tool_name"],
    template="""
You are a cybersecurity expert analyzing the results of a penetration testing tool.

Tool Name: {tool_name}

The user has executed the tool manually in a Kali Linux terminal. Below is the raw output from the tool:

---
{tool_output}
---

Based on the output, provide a detailed analysis including:
- A summary of what the tool did
- Any important findings or vulnerabilities
- What these findings mean in practical terms
- Suggested next steps or follow-up tools
- Risk level (Low, Medium, High) if applicable

Be precise, actionable, and technical — but also beginner-friendly if the output is simple.
"""
)

# 🤖 Create LLM chain for analysis
llm_chain = LLMChain(llm=llm, prompt=analysis_prompt)


LM Studio response: I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you have! How can I help you today?


In [ ]:

# 🧠 Memory + Chat mode for follow-ups
chat_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""
You are a cybersecurity assistant. Only respond with factual tools and commands used in penetration testing. Do not make up data.

{history}
Human: {input}
AI:"""
)

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    prompt=chat_prompt,
    verbose=True
)


In [ ]:

# 📂 Load tool output and run analysis
try:
    tool_output = open("output.txt").read()
    tool_name = "nikto"
    print("\n📊 Initial Tool Output Analysis:\n")
    analysis = llm_chain.run(tool_output=tool_output, tool_name=tool_name)
    print(analysis)
except FileNotFoundError:
    print("⚠️ 'output.txt' not found. Skipping tool analysis.")
    tool_output = None

# 💬 Start chat for follow-up questions (command suggestion, deeper recon, etc.)
print("\n💬 You can now ask follow-up questions (e.g., suggest command, deeper scan). Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = conversation.predict(input=user_input)
    print(f"AI: {response}\n")

⚠️ 'output.txt' not found. Skipping tool analysis.

💬 You can now ask follow-up questions (e.g., suggest command, deeper scan). Type 'exit' to quit.



> Entering new ConversationChain chain...
Prompt after formatting:

You are a cybersecurity assistant. Only respond with factual tools and commands used in penetration testing. Do not make up data.


Human: what is your expertise ?
AI:

> Finished chain.
AI: I have expertise in various penetration testing tools and techniques, including:

1. Nmap (Network Mapper) - for network discovery and port scanning.
2. Metasploit - a popular penetration testing framework for exploiting vulnerabilities.
3. Burp Suite - a web application security testing tool for identifying vulnerabilities.
4. Nessus - a vulnerability scanner for identifying weaknesses in systems and applications.
5. Wireshark - a packet analyzer for monitoring network traffic.
6. ArpScan - a tool for identifying and exploiting ARP spoofing vulnerabilities.

I can provide informati

KeyboardInterrupt: 